# TSA Chapter 1: ACF and PACF Patterns

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch1/TSA_ch1_acf_patterns/TSA_ch1_acf_patterns.ipynb)

This notebook demonstrates:
- ACF patterns for different processes: white noise, AR(1), random walk, seasonal
- ACF and PACF patterns for AR(2), MA(2), ARMA(1,1)
- Theoretical ACF decay: exponential, oscillatory, slow decay

In [ ]:
!pip install yfinance matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def generate_ar1(n, phi, sigma=1.0):
    x = np.zeros(n)
    eps = np.random.normal(0, sigma, n)
    for t in range(1, n):
        x[t] = phi * x[t-1] + eps[t]
    return x

def generate_random_walk(n, sigma=1.0, drift=0.0):
    eps = np.random.normal(0, sigma, n)
    return np.cumsum(eps) + drift * np.arange(n)

def generate_white_noise(n, sigma=1.0):
    return np.random.normal(0, sigma, n)

def generate_seasonal(n, period=12, amplitude=1.0):
    return amplitude * np.sin(2 * np.pi * np.arange(n) / period)

In [ ]:
# Chart: ch1_acf_examples
# ACF patterns for different processes
np.random.seed(42)
n = 300
nlags = 25

wn = generate_white_noise(n)
ar1 = generate_ar1(n, phi=0.7, sigma=1.0)
rw = generate_random_walk(n)
seasonal = generate_ar1(n, phi=0.5) + 2 * np.sin(2 * np.pi * np.arange(n) / 12)

fig, axes = plt.subplots(2, 4, figsize=(9, 3.8))

series = [wn, ar1, rw, seasonal]
titles_top = ['White noise', 'AR(1), phi=0.7', 'Random walk', 'Seasonal (s=12)']
colors_s = [COLORS['blue'], COLORS['green'], COLORS['red'], COLORS['purple']]

for i, (s, title, col) in enumerate(zip(series, titles_top, colors_s)):
    axes[0, i].plot(s, color=col, linewidth=0.5)
    axes[0, i].set_title(title, fontsize=8, fontweight='bold')
    if i == 0:
        axes[0, i].set_ylabel(r'$X_t$')
    axes[0, i].axhline(0 if i != 2 else s.mean(), color=COLORS['gray'], linewidth=0.3, linestyle='--')

    acf_vals = acf(s, nlags=nlags)
    axes[1, i].bar(range(len(acf_vals)), acf_vals, color=col, width=0.5, alpha=0.7)
    ci = 1.96 / np.sqrt(n)
    axes[1, i].axhline(ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
    axes[1, i].axhline(-ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
    axes[1, i].set_xlabel('Lag')
    if i == 0:
        axes[1, i].set_ylabel('ACF')
    axes[1, i].set_title(f'ACF: {title}', fontsize=8, fontweight='bold')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_acf_examples')
plt.show()

In [ ]:
# Chart: acf_pacf_examples
# ACF and PACF patterns for AR, MA, ARMA
np.random.seed(42)
n = 500
nlags = 20

# AR(2)
ar2 = np.zeros(n)
eps = np.random.normal(0, 1, n)
for t in range(2, n):
    ar2[t] = 0.6 * ar2[t-1] - 0.2 * ar2[t-2] + eps[t]

# MA(2)
ma2 = np.zeros(n)
for t in range(2, n):
    ma2[t] = eps[t] + 0.7 * eps[t-1] + 0.3 * eps[t-2]

# ARMA(1,1)
arma11 = np.zeros(n)
for t in range(1, n):
    arma11[t] = 0.6 * arma11[t-1] + eps[t] + 0.4 * eps[t-1]

fig, axes = plt.subplots(2, 3, figsize=(8, 4.0))
titles = ['AR(2)', 'MA(2)', 'ARMA(1,1)']
series_list = [ar2, ma2, arma11]
colors_s = [COLORS['blue'], COLORS['red'], COLORS['green']]

for i, (s, title, col) in enumerate(zip(series_list, titles, colors_s)):
    acf_vals = acf(s, nlags=nlags)
    pacf_vals = pacf(s, nlags=nlags)

    axes[0, i].bar(range(len(acf_vals)), acf_vals, color=col, width=0.5, alpha=0.7)
    ci = 1.96 / np.sqrt(n)
    axes[0, i].axhline(ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
    axes[0, i].axhline(-ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
    axes[0, i].set_title(f'ACF: {title}', fontsize=9, fontweight='bold')
    if i == 0:
        axes[0, i].set_ylabel('ACF')

    axes[1, i].bar(range(len(pacf_vals)), pacf_vals, color=col, width=0.5, alpha=0.7)
    axes[1, i].axhline(ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
    axes[1, i].axhline(-ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
    axes[1, i].set_title(f'PACF: {title}', fontsize=9, fontweight='bold')
    axes[1, i].set_xlabel('Lag')
    if i == 0:
        axes[1, i].set_ylabel('PACF')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'acf_pacf_examples')
plt.show()

In [ ]:
# Chart: acf_theoretical
# Theoretical ACF decay patterns
nlags = 20
lags = np.arange(nlags + 1)

fig, axes = plt.subplots(1, 3, figsize=(8, 2.5))

# Exponential decay (phi > 0)
phi_pos = 0.8
acf_exp = phi_pos ** lags
axes[0].bar(lags, acf_exp, color=COLORS['blue'], width=0.5, alpha=0.7)
axes[0].set_title(r'Exponential decay ($\phi = 0.8$)', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel(r'$\rho(h)$')

# Oscillatory decay (phi < 0)
phi_neg = -0.7
acf_osc = phi_neg ** lags
colors_bars = [COLORS['blue'] if v >= 0 else COLORS['red'] for v in acf_osc]
axes[1].bar(lags, acf_osc, color=colors_bars, width=0.5, alpha=0.7)
axes[1].set_title(r'Oscillatory decay ($\phi = -0.7$)', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel(r'$\rho(h)$')

# Slow decay (near unit root)
phi_slow = 0.99
acf_slow = phi_slow ** lags
axes[2].bar(lags, acf_slow, color=COLORS['red'], width=0.5, alpha=0.7)
axes[2].set_title(r'Slow decay ($\phi = 0.99$)', fontsize=9, fontweight='bold')
axes[2].set_xlabel('Lag')
axes[2].set_ylabel(r'$\rho(h)$')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'acf_theoretical')
plt.show()